#### Create a mini project using the PIMA Indian Diabetes dataset to train a neural network classifier. Include preprocessing, training, evaluation, and comparison with atleast two traditional ML models using metrics like accuracy and confusion matrix

In [ ]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import copy
import torch
import torch.nn as nn
from torch.optim import NAdam
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import RandomOverSampler